## Libraries


In [ ]:
# !pip install -r ~/code/benitomartin/FoodScore/requirements.txt

In [1]:
import os
import cv2
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras import layers 
from tensorflow.keras import Model 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image 
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import load_img, img_to_array, to_categorical, image_dataset_from_directory
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from tensorflow.keras import losses
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import MeanIoU


import pickle

## Data import

In [4]:
coord = pd.DataFrame()

for i in range(1, 101, 1):
    path = f"../raw_data/UECFOOD100/{i}"
    data = pd.read_csv(f"{path}/bb_info.txt", sep=' ', header=0, index_col="img")
    data_df = pd.DataFrame(data)
    data_df["label"] = i
    coord = pd.concat([coord, data_df])
    


In [5]:
coord = coord.reset_index()

### DataFrame with label and coordinates

In [6]:
coord.shape

(14611, 6)

In [7]:
coord.head()

,img,x1,y1,x2,y2,label
0,1,0,143,370,486,1
1,2,20,208,582,559,1
2,3,2,110,243,410,1
3,4,0,237,286,536,1
4,5,8,28,761,585,1


In [8]:
coord = coord.rename(columns={"img": "img_name"})

In [9]:
coord.head()

,img_name,x1,y1,x2,y2,label
0,1,0,143,370,486,1
1,2,20,208,582,559,1
2,3,2,110,243,410,1
3,4,0,237,286,536,1
4,5,8,28,761,585,1


### Rescaling and Normalization

In [10]:
# function to normalize bounding box

def normalize_bbox(row):
    # Read in the image and get its dimensions
    image_path = f"../raw_data/UECFOOD100/{(row['label'])}/{(row['img_name'])}.jpg"
    image = cv2.imread(image_path)
    height, width = image.shape[:2]
    
    # Normalize the coordinates
    x1_norm = row['x1'] / width
    y1_norm = row['y1'] / height
    x2_norm = row['x2'] / width
    y2_norm = row['y2'] / height
    
    # Return normalized coordinates
    return pd.Series({'x1_norm': x1_norm, 'y1_norm': y1_norm, 'x2_norm': x2_norm, 'y2_norm': y2_norm})

# Apply the normalize_bbox function to each row of the DataFrame
normalized_bbox_df = coord.apply(normalize_bbox, axis=1)

# Concatenate the original DataFrame with the new normalized DataFrame
rescaled_coord = pd.concat([coord, normalized_bbox_df], axis=1).drop(columns=['x1', 'y1','x2','y2'])


In [11]:
rescaled_coord.head()

,img_name,label,x1_norm,y1_norm,x2_norm,y2_norm
0,1,1,0.0000,0.238333,0.46250,0.810000
1,2,1,0.0250,0.346667,0.72750,0.931667
2,3,1,0.0025,0.183333,0.30375,0.683333
3,4,1,0.0000,0.395000,0.35750,0.893333
4,5,1,0.0100,0.046667,0.95125,0.975000


### add image paths

In [12]:
list_paths = [f"../raw_data/UECFOOD100/{int(row['label'])}/{int(row['img_name'])}.jpg" for _, row in coord.iterrows()]


In [13]:
rescaled_coord["paths"] = pd.DataFrame(list_paths).copy()

In [14]:
rescaled_coord.head()

,img_name,label,x1_norm,y1_norm,x2_norm,y2_norm,paths
0,1,1,0.0000,0.238333,0.46250,0.810000,../raw_data/UECFOOD100/1/1.jpg
1,2,1,0.0250,0.346667,0.72750,0.931667,../raw_data/UECFOOD100/1/2.jpg
2,3,1,0.0025,0.183333,0.30375,0.683333,../raw_data/UECFOOD100/1/3.jpg
3,4,1,0.0000,0.395000,0.35750,0.893333,../raw_data/UECFOOD100/1/4.jpg
4,5,1,0.0100,0.046667,0.95125,0.975000,../raw_data/UECFOOD100/1/5.jpg


In [15]:
rescaled_coord.to_csv('rescaled_coord.csv')

### balancing Dataset

In [16]:
def rebalancing(df: pd.DataFrame, classes: list, av_number: int = 110, random_state: int = 1) -> pd.DataFrame:
    df_new = df.copy()
    for class_ in classes:
        class_df = df_new[df_new['label'] == class_]
        class_count = len(class_df)
        if class_count > av_number:
            drop_indices = np.random.choice(class_df.index, class_count - av_number, replace=False)
            df_new = df_new.drop(drop_indices)
        else:
            pass
    return df_new

In [17]:
classes = list(set(rescaled_coord.label))

In [18]:
df = rebalancing(rescaled_coord, classes, av_number= 50, random_state=1)

In [19]:
rescaled_coord[rescaled_coord['label']==100].shape

(104, 7)

In [20]:
df[df['label']==100].shape

(50, 7)

### load downscaled pictures into array

In [21]:
from tqdm.auto import tqdm

/Users/katharinarolfs/.pyenv/versions/3.10.6/envs/FoodScore/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
df.head()

,img_name,label,x1_norm,y1_norm,x2_norm,y2_norm,paths
0,1,1,0.0000,0.238333,0.462500,0.810000,../raw_data/UECFOOD100/1/1.jpg
10,12,1,0.1175,0.250000,0.836250,0.945000,../raw_data/UECFOOD100/1/12.jpg
26,30,1,0.0140,0.232000,0.406000,0.666667,../raw_data/UECFOOD100/1/30.jpg
80,4300,1,0.3500,0.408333,0.815625,0.925000,../raw_data/UECFOOD100/1/4300.jpg
104,5075,1,0.3260,0.360000,1.000000,0.992000,../raw_data/UECFOOD100/1/5075.jpg


In [23]:
df_shuffled = df.sample(frac=1, random_state=42)
df_shuffled.head()

,img_name,label,x1_norm,y1_norm,x2_norm,y2_norm,paths
5049,2997,31,0.137500,0.012500,0.945312,0.791667,../raw_data/UECFOOD100/31/2997.jpg
8248,5271,52,0.032941,0.037618,1.000000,1.000000,../raw_data/UECFOOD100/52/5271.jpg
8382,5395,54,0.031111,0.156805,1.000000,0.849112,../raw_data/UECFOOD100/54/5395.jpg
3619,2101,22,0.275000,0.158333,0.956250,0.941667,../raw_data/UECFOOD100/22/2101.jpg
2552,1413,15,0.000000,0.000000,1.000000,1.000000,../raw_data/UECFOOD100/15/1413.jpg


In [85]:
#df_shuffled.to_csv('df_shuffled.csv')

In [24]:
color_order = "BGR"
dims = (224,224)

images = np.empty((len(df_shuffled), dims[0], dims[1], 3), dtype=np.float32)

for i, path in enumerate(tqdm(df_shuffled.paths.values)):
    img = cv2.imread(path)
    img = cv2.resize(img, dims, interpolation=cv2.INTER_AREA)
    if color_order == "RGB":
        img = img[:,:,::-1]
    images[i, :, :, :] = img/255

100%|██████████████████████████████████████| 5000/5000 [00:14<00:00, 344.50it/s]


In [25]:
labels = np.array(df_shuffled.label)
bboxes = np.array(df_shuffled[['x1_norm','y1_norm','x2_norm','y2_norm']], dtype="float32")
paths = np.array(df_shuffled.paths)

In [26]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [27]:
if len(lb.classes_) == 2:
    print("two classes")
    labels = to_categorical(labels)

In [28]:
len(set(df_shuffled.label))

100

In [29]:
tvImages, testImages,tvLabels, testLabels,tvBBoxes, testBBoxes,tvPaths, testPaths=\
train_test_split(images,
                 labels,
                 bboxes,
                 paths,
                 test_size=0.20,
                 random_state=42)

In [30]:
trainImages, valImages,trainLabels, valLabels,trainBBoxes, valBBoxes, trainPaths, valPaths=\
train_test_split(tvImages,
                 tvLabels,
                 tvBBoxes,
                 tvPaths,
                 test_size=0.20,
                 random_state=42)

## Model

In [31]:
vgg = VGG16(weights="imagenet",
            include_top=False,
            input_tensor=layers.Input(shape=(224, 224, 3)))


vgg.trainable = False

flatten = vgg.output
flatten = layers.Flatten()(flatten)

bboxHead = layers.Dense(128, activation="relu")(flatten)
bboxHead = layers.Dense(64, activation="relu")(bboxHead)
bboxHead = layers.Dense(32, activation="relu")(bboxHead)
bboxHead = layers.Dense(4, activation="sigmoid", name="bounding_box")(bboxHead)

softmaxHead = layers.Dense(512, activation="relu")(flatten)
softmaxHead = layers.Dropout(0.5)(softmaxHead)
softmaxHead = layers.Dense(512, activation="relu")(softmaxHead)
softmaxHead = layers.Dropout(0.5)(softmaxHead)
softmaxHead = layers.Dense(len(set(df_shuffled.label)), activation="softmax", name="class_label")(softmaxHead)


In [34]:
model = Model(
    inputs=vgg.input,
    outputs=(bboxHead, softmaxHead))

In [35]:
losses = {
    "class_label": 'categorical_crossentropy',
    "bounding_box": "mse"
}

In [36]:
lossWeights = {
    "class_label": 1.0,
    "bounding_box": 1.0
}

In [37]:
trainTargets = {
    "class_label": trainLabels,
    "bounding_box": trainBBoxes
}

In [38]:
testTargets = {
    "class_label": testLabels,
    "bounding_box": testBBoxes
}

In [39]:
valTargets = {
    "class_label": valLabels,
    "bounding_box": valBBoxes
}

In [40]:
metrics = {
    "class_label": "categorical_accuracy",
    "bounding_box": MeanIoU(num_classes=len(set(df_shuffled.label)))
}

In [41]:
opt = Adam(0.001)


model.compile(loss=losses, 
              optimizer=opt, 
              metrics=metrics, 
              loss_weights=lossWeights)

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 224, 224, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 224, 224, 64  36928       ['block1_conv1[0][0]']           
                                )                                                             

In [42]:
es = EarlyStopping(monitor = 'loss',
                   patience = 5,
                   verbose = 0,
                   restore_best_weights = True)

In [43]:
history = model.fit(
    trainImages,
    trainTargets,
    validation_data=(valImages, valTargets),
    batch_size=32,
    epochs=1000,
    verbose=1,
    callbacks = [es],
    )

Epoch 1/1000


2023-03-23 11:48:40.302693: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


  5/100 [>.............................] - ETA: 4:49 - loss: 7.8489 - bounding_box_loss: 0.0575 - class_label_loss: 7.7914 - bounding_box_mean_io_u: 0.4592 - class_label_categorical_accuracy: 0.0125

KeyboardInterrupt: 

## Save model

In [ ]:
# VGG16

pickle.dump(model, open('vgg16_100classes_50imgs.pkl', 'wb'))